# 03_Gold_Embeddings

## Que es Sentence Transformers? 

Libreria que convierte **texto a vectores** (embeddings). 

## Como funciona?

ENTRADA (texto): "startup de delivery que fallo por competencia con Uber Eats"

SALIDA (vector de 384 numeros): [0.023, -0.156, 0.089, 0.234, ... , -0.045]

Code

## Para que sirve? 

Los vectores capturan el **significado** del texto.  Textos similares = vectores cercanos.

| Texto | Vector cercano a...  |
|-------|---------------------|
| "fallo por falta de dinero" | Texto B |
| "se quedo sin presupuesto" | Texto A |
| "competencia de Google" | Ninguno (diferente) |

## En nuestro proyecto

1. Convertimos cada startup fallida a un vector
2.  Cuando el usuario describe SU startup, la convertimos a vector
3. Buscamos startups fallidas con vectores cercanos
4. El modelo predice basandose en muertes similares

In [3]:
%pip install sentence-transformers -q

print("Instalacion completada!")

Note: you may need to restart the kernel to use updated packages.
Instalacion completada!


In [4]:
from sentence_transformers import SentenceTransformer
import pandas as pd

# Cargar modelo (pequeño y rapido)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Cargar datos preparados
gold_path = "/data/gold/"
df = pd. read_csv(gold_path + "failures_for_embeddings.csv")

print("MODELO CARGADO:")
print("=" * 50)
print(f"Modelo: all-MiniLM-L6-v2")
print(f"Dimension del embedding: {model.get_sentence_embedding_dimension()}")
print(f"\nDATOS:")
print(f"Startups a procesar: {len(df)}")

KeyboardInterrupt: 

## Que es generar embeddings? 

Convertir texto a numeros que la IA puede entender y comparar. 

### Antes vs Despues

| Antes (texto) | Despues (vector) |
|---------------|------------------|
| "company aira health.  sector health care. causes: no_budget, competition" | [0.023, -0.156, 0.089, ...  ] (384 numeros) |

### Por que 384 numeros?

El modelo **all-MiniLM-L6-v2** comprime el significado del texto en 384 dimensiones.  Cada numero representa una "caracteristica" del texto (sector, causas, contexto, etc). 

### Para que sirve? 

Busqueda por similitud: vectores cercanos = startups similares = mismas causas de muerte. 

Tu startup: [0.02, -0.15, 0.09, ...]

Aira Health: [0.023, -0.156, 0.089, ... ] --> MUY SIMILAR! 

✓ Otro startup: [0. 98, 0. 45, -0. 72, ...] --> MUY DIFERENTE ✗

In [0]:
import numpy as np

# Obtener lista de textos
textos = df['text'].tolist()

print("Generando embeddings para 409 startups...")
print("=" * 50)

# Generar embeddings (esto tarda unos segundos)
embeddings = model.encode(textos, show_progress_bar=True)

print(f"\nEMBEDDINGS GENERADOS!")
print(f"Shape: {embeddings. shape}")
print(f"Cada startup tiene un vector de {embeddings. shape[1]} dimensiones")

Generando embeddings para 409 startups...


Batches:   0%|          | 0/13 [00:00<?, ?it/s]


EMBEDDINGS GENERADOS!
Shape: (409, 384)
Cada startup tiene un vector de 384 dimensiones


In [0]:
# Agregar embeddings al dataframe
df['embedding'] = embeddings. tolist()

# Guardar como CSV
df.to_csv(gold_path + "failures_with_embeddings.csv", index=False)

# Guardar embeddings como numpy (mas eficiente para cargar)
np.save(gold_path + "embeddings.npy", embeddings)

print("GUARDADO:")
print("=" * 50)
print(f"CSV: failures_with_embeddings.csv")
print(f"NPY: embeddings. npy (formato numpy)")
print(f"\nArchivos en: {gold_path}")

GUARDADO:
CSV: failures_with_embeddings.csv
NPY: embeddings. npy (formato numpy)

Archivos en: /Workspace/startup-death-oracle/data/gold/


## Prueba de busqueda por similitud

Esta celda simula lo que hara el oraculo cuando un usuario describa su startup. 

### Paso a paso:

Vamos a generar una startup de prueba

1.  **Texto de prueba**: Simulamos un usuario que describe su startup
"food delivery app competing with uber eats, raised 5 million, high costs"

Code

2. **Generar embedding**: Convertimos ese texto a un vector de 384 numeros

3. **Calcular similitud**: Comparamos ese vector con los 409 vectores de startups muertas usando *cosine similarity* (mide que tan "cercanos" estan dos vectores)

4. **Top 3**: Mostramos las 3 startups fallidas mas similares

### Resultado esperado:

Tu startup: food delivery app competing with uber eats...

Startups similares que murieron:

SpoonRocket Sector: Food & Beverage Similitud: 87%

Maple Sector: Food & Beverage
Similitud: 82%

Munchery Sector: Food & Beverage Similitud: 79%

Code

Esto es el corazon del proyecto: encontrar muertes similares para predecir la del startup. 

In [0]:
from sklearn.metrics. pairwise import cosine_similarity

# Texto de prueba (simula una startup nueva)
startup_usuario = "food delivery app competing with uber eats, raised 5 million, high costs"

# Generar embedding para el texto de prueba
embedding_usuario = model. encode([startup_usuario])

# Calcular similitud con todas las startups
similitudes = cosine_similarity(embedding_usuario, embeddings)[0]

# Obtener top 3 mas similares
top_indices = similitudes.argsort()[-3:][::-1]

print("PRUEBA DE SIMILITUD:")
print("=" * 50)
print(f"Tu startup: {startup_usuario}")
print("\nStartups similares que murieron:")
print("-" * 50)

# Recorre los indices de las 3 startups mas similares
# enumerate agrega un contador (i) que empieza en 1
for i, idx in enumerate(top_indices, 1):
    
    # Imprime el numero y nombre de la startup
    # df.iloc[idx] accede a la fila por su posicion
    print(f"\n{i}. {df.iloc[idx]['name']}")
    
    # Imprime el sector de esa startup
    print(f"   Sector: {df. iloc[idx]['sector']}")
    
    # Imprime la similitud con 2 decimales
    # Ejemplo: 0.87 significa 87% similar
    print(f"   Similitud: {similitudes[idx]:.2f}")

PRUEBA DE SIMILITUD:
Tu startup: food delivery app competing with uber eats, raised 5 million, high costs

Startups similares que murieron:
--------------------------------------------------

1. Sprig
   Sector: Accommodation and Food Services
   Similitud: 0.57

2. Sprig
   Sector: Health Care
   Similitud: 0.50

3. Chef'd
   Sector: Accommodation and Food Services
   Similitud: 0.46


Hemos encontrado startups similares

La descripcion ("food delivery app competing with uber eats") encontro startups de comida que murieron. Funciona!

In [0]:
print("03_GOLD_EMBEDDINGS COMPLETADO!")
print("=" * 50)
print(f"""
MODELO: all-MiniLM-L6-v2
EMBEDDINGS: 409 startups x 384 dimensiones

ARCHIVOS GENERADOS:
├── failures_with_embeddings.csv
└── embeddings. npy

SIGUIENTE PASO:
Construir el oraculo con RAG para generar
predicciones personalizadas. 
""")

03_GOLD_EMBEDDINGS COMPLETADO!

MODELO: all-MiniLM-L6-v2
EMBEDDINGS: 409 startups x 384 dimensiones

ARCHIVOS GENERADOS:
├── failures_with_embeddings.csv
└── embeddings. npy

SIGUIENTE PASO:
Construir el oraculo con RAG para generar
predicciones personalizadas. 



## Modelo utilizado

Hemos usado el modelo **all-MiniLM-L6-v2** para convertir el texto de cada startup en un vector de 384 dimensiones. 

### Por que este modelo?

| Caracteristica | Valor |
|----------------|-------|
| Tamaño | 80 MB (ligero) |
| Velocidad | Rapido |
| Dimension | 384 |
| Creador | Microsoft |

### Alternativas consideradas

| Modelo | Dimension | Tamaño | Por que no |
|--------|-----------|--------|------------|
| all-mpnet-base-v2 | 768 | 420 MB | Mas lento, innecesario para 409 registros |
| text-embedding-ada-002 | 1536 | API externa | Costo por uso, dependencia de OpenAI |
| all-MiniLM-L6-v2 | 384 | 80 MB | **Elegido: equilibrio entre calidad y velocidad** |

### Resultado

Cada startup ahora tiene un vector que captura su significado.  Startups similares tendran vectores cercanos, permitiendo al oraculo encontrar "muertes similares". 